In [1]:
## add ignore warnings for now, will remove and debug once full algorithm is complete
import warnings
warnings.filterwarnings("ignore")

## import packages/libraries
from time import perf_counter, clock_gettime_ns, CLOCK_REALTIME
import numpy as np
from multiprocessing import Pool, cpu_count
from itertools import product
import sys
import sqlite3

## append filepath to allow files to be called from within project folder
sys.path.append('/home/gerard/Desktop/capstone_project/patoms')
sys.path.append('/home/gerard/Desktop/capstone_project')

## call locally created functions
from snapshot_2d_pattern_v4 import patoms2d
from snapshot_3d_pattern_v6 import patoms3d
from pattern_2d_compare_v3 import pattern_compare_2d
from pattern_3d_compare_v4 import pattern_compare_3d
from get_2d_table_names_v0 import table_names_2d
from updating_ref_table_v0 import update_ref_table
from patom_to_table_v0 import patom_to_table_func

In [2]:
## start timer to asses how long process takes
s = perf_counter()

## create in memory 2d database
con2d = sqlite3.connect(":memory:")
cur2d = con2d.cursor()

## create (if not exists) and connect to 3D database
# con3d = sqlite3.connect("db3d.db")
# cur3d = con3d.cursor()

## create test data for algorithm development
#np.random.seed(42)
fps30 = [1] * 2

dist_sim_threshold = 0.85
centroid_sim_threshold_x = 0.85
centroid_sim_threshold_y = 0.85

In [3]:
for i in fps30:
    rand_array = np.random.random((i, 720, 1280))
    z_len = rand_array.shape[0]
    y_len = rand_array.shape[1]
    x_len = rand_array.shape[2]
    # ingest data frame by frame
    for frame in range(rand_array.shape[0]):
        #################################################################################
        ####################### FIRST TASK: FIND PATTERNS IN FRAME ######################
        #################################################################################
        frame_patoms = patoms2d(x_len, y_len, rand_array[frame,:,:], frame)
        # patom = [[norm_x, norm_y], [pattern_centroid_x, pattern_centroid_y], patom_ind, frame_ind, patom_time]
        # patom[i][[0][0]: list of x_pos, patom[i][[0][1]: list of y_pos, patom[i][[1][0]: x_cent, patom[i][[1][1]: y_cent, patom[i][[2]: patom_ind, patom[i][[3]: frame_ind, patom[i][[4]: patom_time,
        num_patoms = len(frame_patoms)
        
        #############################################################################################################################################################
        ########## THIRD TASK: STORE NEW PATTERNS IN EMPTY TABLES AND ADD OLD (PATTERNS SIMILAR TO PREVIOUSLY RECEIVED DATA) IN THEIR RESPECTIVE TABLES #############
        #############################################################################################################################################################
        ## get pattern tables and find empty and non-empty
        tables = table_names_2d(cur2d) #return [empty_nonref, nonempty_nonref, empty_ref, nonempty_ref]
        empty_nonref_tables = tables[0]
        nonempty_nonref_tables = tables[1]
        empty_ref_tables = tables[2]
        nonempty_ref_tables = tables[3]

        # if there are non-empty reference tables then loop through tables and compare reference pattern against newly acquired patterns
        # if simialr patterns exist add to relevant existing table, if no similar patterns exist write new patterns to empty table
        if nonempty_ref_tables:
            # convert non-empty ref tables back into format for pattern comparing
            working_ref_tables = []
            for i in nonempty_ref_tables:
                table = cur2d.execute(f"select * from {i};").fetchall()
                # patom = [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, patom_ind, frame_ind, patom_time]
                norm_x = [x[0] for x in table]; norm_y = [x[1] for x in table]; pattern_centroid_x = [x[2] for x in table][0]; pattern_centroid_y = [x[3] for x in table][0]
                working_ref_tables.append([norm_x, norm_y, pattern_centroid_x, pattern_centroid_y])
            # compare new patterns against existing reference patterns, if patterns do not match ref pattern store in new table
            patom_ref_indexes = list(product(range(num_patoms), range(len(working_ref_tables))))
            with Pool(processes=cpu_count()) as pool:
                items = [(frame_patoms[i[0]][0], frame_patoms[i[0]][1], frame_patoms[i[0]][2], frame_patoms[i[0]][3],\
                        working_ref_tables[i[1]][0], working_ref_tables[i[1]][1], working_ref_tables[i[1]][2], working_ref_tables[i[1]][3], i) for i in patom_ref_indexes]
                ## function outputs ind value of the patom_indexes list, the centroid and distance similarity measures
                res = pool.starmap(pattern_compare_2d, items)
                # res output: [1.0, (0.05407046509274386, 0.03361332388786884)]
                ## loop through the output of the comparison function
                for ix, i in enumerate(res):
                    ## pass if its the same patom in the frame being compared against itself
                    if patom_ref_indexes[ix][0] == patom_ref_indexes[ix][1]:
                        pass
                    ## check if compared patterns fall within similarity threshold values, if they do store in table
                    elif (i[1][0] >= centroid_sim_threshold_x) and (i[1][1] >= centroid_sim_threshold_y) and (i[0] >= dist_sim_threshold):
                        # add patom to relevant pattern table get name of ref table to get data table name
                        print('similar')
                        table_num = nonempty_ref_tables[patom_ref_indexes[ix][1]][-6:] 
                        new_patom = frame_patoms[patom_ref_indexes[ix][0]]
                        patom_to_table = patom_to_table_func(new_patom)
                        cur2d.executemany(f"INSERT INTO pat_2d_{table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                        con2d.commit()
                        # patom = [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, patom_ind, frame_ind, patom_time]
                    else:
                        # if patom is not matched then create new pattern & ref tables, get max table name and then create next new one
                        next_table_num = int(max([table for (table,) in cur2d.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE '%ref%';").fetchall()])[-6:]) + 1
                        next_table_num = str(next_table_num).zfill(6)
                        new_patom = frame_patoms[patom_ref_indexes[ix][0]]
                        patom_to_table = patom_to_table_func(new_patom)
                        cur2d.execute(f"CREATE TABLE pat_2d_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);")
                        cur2d.execute(f"CREATE TABLE pat_2d_ref_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);")
                        cur2d.executemany(f"INSERT INTO pat_2d_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                        cur2d.executemany(f"INSERT INTO pat_2d_ref_{next_table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                        con2d.commit()
                        # patom = [norm_x, norm_y, pattern_centroid_x, pattern_centroid_y, patom_ind, frame_ind, patom_time]

        # should only be a one time activity       
        # if there are no non-empty reference tables or if none of the patterns are similar to ref patoms then write patterns to new tables
        else:
            # create new pattern/data & ref tables for new patoms
            for i in range(num_patoms):
                table_num = str(i).zfill(6)
                patom = frame_patoms[i]
                patom_to_table = patom_to_table_func(patom)
                cur2d.execute(f"CREATE TABLE pat_2d_{table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);")
                cur2d.execute(f"CREATE TABLE pat_2d_ref_{table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time);")
                cur2d.executemany(f"INSERT INTO pat_2d_{table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                cur2d.executemany(f"INSERT INTO pat_2d_ref_{table_num}(x_pos_dist, y_pos_dist, x_cent, y_cent, patom_ind, frame_ind, patom_time) VALUES (?,?,?,?,?,?,?)", patom_to_table)
                con2d.commit()




In [4]:
patom_table_names = cur2d.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE '%ref%';").fetchall()
patom_tables = [name for (name,) in patom_table_names]
for (name,) in patom_table_names:
    print(cur2d.execute(f"select count(distinct patom_time) from {name};").fetchone())

# con2d.close()

(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
(1,)
